# Estimate Coin

Taken from Dustin Tran's example on the Edward Github https://github.com/blei-lab/edward/blob/master/examples/beta_bernoulli.py

In [2]:
import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import seaborn as sns

from edward.models import Bernoulli, Beta, Empirical

Set random seed

In [3]:
ed.set_seed(42)

Example data

In [4]:
#x_data = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1])
x_data = np.random.binomial(1, 0.25, 100)

Describe the model in terms of the distributions of its variables. The Bernoulli distribution is defined in terms of sample shape, so it must be in line with the shape of our data

In [5]:
# MODEL
p = Beta(1.0, 1.0)
x = Bernoulli(probs=p, sample_shape=100)

## Gibbs sampling

Define a tensorflow variable that takes the samples tf.Variable("initialvalue", name) (https://www.tensorflow.org/api_docs/python/tf/Variable)

In [19]:
qp_gibbs = Empirical(params=tf.Variable(tf.zeros(500) # Number of iterations?
    #,initializer=tf.constant_initializer(0.5) # Initialize the first state of the chain?
))

In [20]:
inference_gibbs = ed.Gibbs({p: qp_gibbs}, data={x: x_data})

/Users/hauptjoh/anaconda/envs/edward/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
/Users/hauptjoh/anaconda/envs/edward/lib/python3.6/site-packages/edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


In [21]:
inference_gibbs.run()

500/500 [100%] ██████████████████████████████ Elapsed: 1s | Acceptance Rate: 1.000


In [17]:
# CRITICISM
# exact posterior has mean 0.25 and std 0.12
sess = ed.get_session()
sess.run([qp_gibbs.mean(), qp_gibbs.stddev()])

[0.24491891, 0.04274016]

## Metropolis Hastings

The Metropolis Hastings algorithm works by changing the inital state of the chain based on a *proposal distribution* and than either accepting or rejecting the adjustment. 

"This proposal distribution randomly perturbs the current state of the chain, and then either accepts or rejects the pertubed value. Algorithms of this form are called “Random-walk Metropolis algorithm.” See http://www.mit.edu/~ilkery/papers/MetropolisHastingsSampling.pdf

**TODO**: Understand why the beta distribution with 3.0, 9.0 is chosen as default proposal distribution here. Might just be a reasonable (prior?) shape for the empirical distirbution which is also centered at about 0.25

In [5]:
proposal_p = Beta(3.0, 9.0)

Empirical() defines the expected empirical distribution after sampling. It defines the collection of samples *params* in terms of the internal tf variable name, the number of samples (**TODO**: What about early stopping?) and the initial state.

tf.get_variable() gets an existing variable with these parameters or creates a new one.     

**TODO**: Learn more about the tensorflow variable management. How to overwrite the tf variable name when code is run again?

In [6]:
# INFERENCE
qp = Empirical(params=tf.get_variable(
    name = "qp/params",  # Defines name that is internally used?
    shape = [1000], # Number of iterations for the Metropolis Hasting algorithm? Higher number slows down inference
    initializer=tf.constant_initializer(0.5) # Initialize the first state of the chain?
))

Run the actual inference alogrithm

Run the inference algorithm by specifying the data to be used.

**TODO**: Understand relation between Empirical() and data here.

In [7]:
inference = ed.MetropolisHastings(latent_vars = {p: qp}, proposal_vars = {p: proposal_p}, data={x: x_data})

/Users/hauptjoh/anaconda/envs/edward/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
/Users/hauptjoh/anaconda/envs/edward/lib/python3.6/site-packages/edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


In [8]:
inference.run()

100000/100000 [100%] ██████████████████████████████ Elapsed: 114s | Acceptance Rate: 0.412


"Criticism" of the model, i.e. analysis of its estimates and confidence.

Tensorflow defines its environment in a session. 

**TODO**: Better understand variable access in sessions in tensorflow 

In [9]:
# CRITICISM
# exact posterior has mean 0.25 and std 0.12
sess = ed.get_session()
mean, stddev = sess.run([qp.mean(), qp.stddev()])
print("Inferred posterior mean:")
print(mean)
print("Inferred posterior stddev:")
print(stddev)

Inferred posterior mean:
0.24465695
Inferred posterior stddev:
0.04264308


In [18]:
x_post = ed.copy(x, {p: qp})
tx_rep, tx = ed.ppc(
  lambda xs, zs: tf.reduce_mean(tf.cast(xs[x_post], tf.float32)),
  data={x_post: x_data})
ed.ppc_stat_hist_plot(
  tx[0], tx_rep, stat_name=r'$T \equiv$mean', bins=20)
plt.show()

TypeError: probs must having floating type.